In [23]:
import pandas as pd
import re


ratings_movies = pd.read_csv('data/ratings_movies.csv', sep=',')

ratings_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100836 non-null  int64  
 1   userId      100836 non-null  int64  
 2   movieId     100836 non-null  int64  
 3   rating      100836 non-null  float64
 4   date        100836 non-null  object 
 5   title       100836 non-null  object 
 6   genres      100836 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 5.4+ MB


In [24]:
#библиотека для регулярных выражений

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
    
#Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска?    
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
display(ratings_movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


None

In [25]:
#Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
# В качестве ответа запишите название этого фильма без указания года его выпуска.
    # Подсказка (1 из 4): Произведите фильтрацию по условию «год выпуска».
    # Подсказка (2 из 4): На отфильтрованной таблице произведите группировку по названию фильма(title) и посчитайте среднее по столбцу rating для каждого фильма.
    # Подсказка (3 из 4): Отсортируйте результат в порядке возрастания.
    # Подсказка (4 из 4): Индекс первой строки результирующей таблицы будет являться ответом.
mask = ratings_movies['year_release'] == 1999
ratings_movies[mask].groupby('title')['rating'].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [26]:
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).
#     Подсказка (1 из 4): Произведите фильтрацию по условию «год выпуска».
#     Подсказка (2 из 4): Сгруппируйте отфильтрованную таблицу по жанрам и найдите средний рейтинг для каждого жанра.
#     Подсказка (3 из 4): Отсортируйте результат по возрастанию.
#     Подсказка (4 из 4): Индекс первой строки результирующей таблицы будет являться ответом.

mask1 = ratings_movies['year_release'] == 2010
ratings_movies[mask1].groupby('genres')['rating'].mean().sort_values(ascending=True)

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [27]:
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов?
# В качестве ответа запишите идентификатор этого пользователя.
#     Подсказка (1 из 3): Сгруппируйте таблицу по пользователям и найдите число уникальных жанров для каждого пользователя.
#     Подсказка (2 из 3): Отсортируйте результат по убыванию.
#     Подсказка (3 из 3): Индекс первой строки результирующей таблицы будет являться ответом.

ratings_movies.groupby('userId')['genres'].nunique().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

In [28]:
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
# В качестве ответа укажите идентификатор этого пользователя.
# Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), 
# можно воспользоваться методом agg() на сгруппированных данных.
#     Подсказка (1 из 4): Сгруппируйте таблицу по пользователям (userId).
#     Подсказка (2 из 4): Для каждого пользователя рассчитайте количество выставленных им оценок и средний рейтинг.
#     Подсказка (3 из 4): Отсортируйте таблицу по возрастанию количества оценок и по убыванию среднего рейтинга.
#     Подсказка (4 из 4): Индекс первой строки результирующей таблицы будет являться ответом.

ratings_movies.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(['count', 'mean'], ascending=[True, False])


,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [29]:
# Найдите сочетание жанров (genres) за 2018 год, которое имеет наибольший средний рейтинг (среднее по столбцу rating),
# и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).
#     Подсказка (1 из 6): Произведите фильтрацию по условию «год выпуска 2018».
#     Подсказка (2 из 6): Сгруппируйте отфильтрованную таблицу по жанрам и найдите средний рейтинг, 
#                         а также количество выставленных оценок с помощью метода agg().
#     Подсказка (3 из 6): Сохраните сгруппированную таблицу.
#     Подсказка (4 из 6): Произведите фильтрацию полученной таблицы по условию «количество оценок > 10».
#     Подсказка (5 из 6): Отсортируйте результат по столбцу со средним значением по убыванию.
#     Подсказка (6 из 6): Индекс первой строки результирующей таблицы будет являться ответом.

mask2 = ratings_movies['year_release'] == 2018
r_t = ratings_movies[mask2].groupby('genres')['rating'].agg(['mean', 'count'])
display(r_t[r_t['count'] > 10].sort_values(by='mean', ascending=False))


,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [61]:
# Добавьте в таблицу новый признак year_rating — год выставления оценки. 
# Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра. 
# Выберите верные варианты ответа, исходя из построенной таблицы:
    # Подсказка (1 из 3): Переведите признак date в формат datetime.
    # Подсказка (2 из 3): Выделите признак года выставления оценки с помощью атрибута аксессора dt year,
    #                     результат занесите в столбец year_rating.
    # Подсказка (3 из 3): Составьте сводную таблицу, индексами которой являются годы выхода фильмов (year_rating), 
    #                     столбцами — жанры (genres), а в качестве значений выступают средние оценки фильмов.

ratings_movies['date'] = pd.to_datetime(ratings_movies['date'],dayfirst=True)
ratings_movies['year_rating'] = ratings_movies['date'].dt.year
pivot_r = ratings_movies.pivot_table(values='rating', index='year_rating',columns='genres', aggfunc='mean', fill_value=0)
pivot_r
